In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import torchvision.models as models

import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets

import torchvision.transforms as transforms

import numpy as np
import pandas as pd
import cv2
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

C:\Users\krish\AppData\Roaming\Python\Python37\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\krish\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\krish\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [48]:
categories = ('rock','paper','scissors')

In [126]:
def create_training_test_data(path):
    datadir = path
    
    training_data = []
    test_data = []
    for category in categories:
        classes = []
        path = datadir+ "/" + category
        class_num = categories.index(category)
        print(categories.index(category))
        print("Number of images in "+category + " = " + str(len(os.listdir(path))))
        for img in os.listdir(path):
            new_array = cv2.imread((path+"/"+img))
            npimg = np.asarray(new_array)
            #print(new_array.shape)
            class_numb = [class_num]
            classes.append([npimg,class_num])
        random.shuffle(classes)
        for i in range(0,int(0.2*len(classes))):
            test_data.append(classes[i])
        for i in range(int(0.2*len(classes)),len(classes)):
            training_data.append(classes[i])
            
    return training_data,test_data

In [127]:
training_data,validation_test_data = create_training_test_data('./image_data')

0
Number of images in rock = 1000
1
Number of images in paper = 1000
2
Number of images in scissors = 1000


In [134]:
print(len(training_data))
print(len(validation_test_data))

2400
600


In [94]:
def create_training_data(path):
    datadir = path
    training_data = []
    test_data = []
    for category in categories:    
        path = datadir + "/" + category
        class_num = categories.index(category)
        print(categories.index(category))
        print("Number of images in "+category + " = " + str(len(os.listdir(path))))
        for img in os.listdir(path):
            new_array = cv2.imread((path+'/'+img))
            npimg = np.asarray(new_array)
            class_numb = [class_num]
            #npimg = transforms(npimg)
            training_data.append([npimg,class_num])
            #print(training_data)
            
    return training_data

In [95]:
training_data = create_training_data('./image_data')

0
Number of images in rock = 500
1
Number of images in paper = 500
2
Number of images in scissors = 500


In [135]:
print(len(training_data))
print(len(validation_test_data))

2400
600


In [140]:
batch_size = 8
validation_size = int(len(validation_test_data)/2)

validation_data,test_data = random_split(validation_test_data,[validation_size,validation_size])
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [139]:
batch_size = 8
full_data = training_data
train_size = int(0.8 * len(full_data))
validationtest_size = int(0.2*len(full_data))
validation_size = int(0.1*len(full_data))
test_size = int(0.1*len(full_data))

train_data, validationtest_data = random_split(full_data, [train_size, validationtest_size])
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False)
validation_data,test_data = random_split(validationtest_data,[validation_size, test_size])
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [144]:
index = {'0': 0,'1':0,'2':0}
for pic,value in training_data:
    index[str(value)] += 1
print(index)

{'0': 800, '1': 800, '2': 800}


In [54]:
def validation(model, val_losses):
    with torch.no_grad():
        model.eval()
        for img,index in validation_loader:
            npimg = (img).numpy()
            new_array = np.transpose(npimg,(0,3,1,2))
            new_array = torch.from_numpy(new_array)
            new_array = new_array.type(torch.FloatTensor)
            outputs = model(new_array.to(device))
            index = index.type(torch.LongTensor)
            val_loss = loss_fn(outputs, index)
            val_losses.append(val_loss.item())
    return val_losses

In [55]:
def train(model,train_losses,device):
  model.train()
  for batch_id,data in enumerate(train_loader):
    inputs,label = data
    opt.zero_grad()

    #-----------------------Preprocessing--------------------------

    npimg = (inputs).numpy()
    new_array = np.transpose(npimg,(0,3,1,2))
    new_array = torch.from_numpy(new_array)
    new_array = new_array.type(torch.FloatTensor)

    #-----------------------Forward--------------------------------

    outputs = model(new_array.to(device))
    loss = loss_fn(outputs,label.to(device))
    train_losses.append(loss.item())

    #-----------------------Backward----------------------------------

    loss.backward()
    opt.step()
    return train_losses

In [56]:
def test(model,device):
  
  model.eval()
  test_loss = 0
  tot = 0
  c = 0
  with torch.no_grad():

    for batch_id,data in enumerate(validation_loader):
      inputs,label = data
      npimg = (inputs).numpy()
      new_array = np.transpose(npimg,(0,3,1,2))
      new_array = torch.from_numpy(new_array)
      new_array = new_array.type(torch.FloatTensor)
      outputs = model(new_array.to(device))
      val,index = torch.max(outputs,1)
      label = label.to(device)
      #print(label,index)
      tot += batch_size
      for i in range(0,len(label)):
        if label[i] == index[i]:
          c+= 1
    print("Accuracy:",c,tot,c/tot)

In [57]:
def print_prediction(model, dataset, device):
    idx = np.random.randint(len(dataset))
    #path = dataset.pathes[idx]
    
    with torch.no_grad():
        model.eval()
        img, target_text = dataset[idx]
        image = img
        new_array = np.transpose(img,(2,0,1))
        new_array = torch.from_numpy(new_array)
        new_array = new_array.type(torch.FloatTensor)
        logits = model(new_array.unsqueeze(0).to(device))

    pred_text = categories[torch.max(logits,1)[1]]
    print("Truth",categories[target_text],"Pred",pred_text)

In [58]:
from torchvision.models.alexnet import alexnet

In [59]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [60]:
from torchvision.models.resnet import resnet18
                    


In [152]:
batch_size = batch_size
max_epoch = 10

#model = Conv1().to(device)

'''
model = alexnet(pretrained = True)
model.classifier[6] = nn.Sequential(
                    nn.Linear(4096,1000,bias = True),
                    nn.BatchNorm1d(1000),
                    nn.ReLU(),
                    nn.Linear(1000,500,bias = True),
                    nn.BatchNorm1d(500),
                    nn.ReLU(),
                    nn.Linear(500,50,bias = True),
                    nn.BatchNorm1d(50),
                    nn.ReLU(),
                    nn.Linear(50,3,bias = True),
                    nn.Softmax(dim= 1)
                    )

model.to(device)
'''

#'''
model = resnet18(pretrained = True)
model.fc = nn.Sequential(
    nn.Dropout(0.4),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Linear(512,3,bias = True),
    nn.Softmax()
)   

model.to(device)
#'''

#model = Conv2().to(device)

loss_fn = nn.CrossEntropyLoss()

#-------------------------------ADADELTA------------------------------------
opt = torch.optim.Adadelta(model.parameters(),lr = 1)

#-------------------------------ADAM-------------------------------------
#opt = torch.optim.Adam(model.parameters(),lr = 0.01)

#--------------------------------SGD-------------------------------------------

#opt = torch.optim.SGD(model.parameters(), lr = 0.1, momentum=0.1,nesterov=True)

In [153]:
train_losses = []
val_losses = []
validation_losses = []

In [156]:
try:
  epoch = 1

  while epoch<=max_epoch:
    print("Epoch",epoch)
    train_losses = train(model,train_losses,device)
    test(model,device)
    print_prediction(model,validation_data,device)
    #plot_loss(epoch, train_losses)
    print("Train Loss",train_losses[-1])
    epoch += 1
    #torch.save(model,"/content/checkpoints/Conv2_e_"+str(epoch)+".pt")
except KeyboardInterrupt:
  pass
    

Epoch 1
Accuracy: 300 304 0.9868421052631579
Truth rock Pred rock
Train Loss 0.5552856922149658
Epoch 2
Accuracy: 300 304 0.9868421052631579
Truth rock Pred rock
Train Loss 0.5567788481712341
Epoch 3
Accuracy: 300 304 0.9868421052631579
Truth paper Pred paper
Train Loss 0.5559231638908386
Epoch 4
Accuracy: 300 304 0.9868421052631579
Truth scissors Pred scissors
Train Loss 0.555047869682312
Epoch 5
Accuracy: 300 304 0.9868421052631579
Truth rock Pred rock
Train Loss 0.5573054552078247
Epoch 6
Accuracy: 300 304 0.9868421052631579
Truth scissors Pred scissors
Train Loss 0.5539582967758179
Epoch 7
Accuracy: 300 304 0.9868421052631579
Truth scissors Pred scissors
Train Loss 0.5546984076499939
Epoch 8
Accuracy: 300 304 0.9868421052631579
Truth paper Pred paper
Train Loss 0.5548917055130005
Epoch 9
Accuracy: 300 304 0.9868421052631579
Truth rock Pred rock
Train Loss 0.5538460612297058
Epoch 10
Accuracy: 300 304 0.9868421052631579
Truth paper Pred paper
Train Loss 0.5537961721420288


In [155]:
torch.save(model.type(torch.float32),'resnet.pt')